In [13]:
import pandas as pd
import numpy as np
# Make sure you have this installed in your environment: pip install thefuzz
from thefuzz import process

In [14]:
# --- 1. LOAD MESSY DATA ---
# Path is relative from the 'posts' directory: up two levels (../../) then down.
try:
    df_wide = pd.read_excel("../files/datasets/Relegation Attendance Churn_copy.xlsx", sheet_name=0, header=0)
except FileNotFoundError:
    print("Error: '../files/datasets/Relegation Attendance Churn_copy.xlsx' not found. Using example data.")
    # Example data in case the file isn't found
    df_wide = pd.DataFrame({
        'Season': ['1992-93'], 'Relegated Team': ['Nottingham Forest'], 'Position': ['Last'],
        'Year Before': ['1991-1992'], 'Year Before Div': ['PL'], 'Year Before Att': '23,721',
        'Attendance': '21,910', 'Year After': ['1993-94'], 'Year After Division': ['Championship'],
        'Attendance year after': '23,051', '2 years after': ['1994-95'], '2 Years After Div': ['PL'],
        'Attendance 2 years after': '23,633'
    })

# --- Polished Cleaning ---
# Find only the attendance columns first
attendance_cols = [c for c in df_wide.columns if "Attendance" in c]

# Now, loop *only* through that smaller list
for col in attendance_cols:
    # Convert *just this column* to string before cleaning
    df_wide[col] = df_wide[col].astype(str).str.replace(',', '').replace('COVID', pd.NA).replace('nan', pd.NA)
    df_wide[col] = pd.to_numeric(df_wide[col], errors='coerce')
# --- End Fix ---

print("Step 1: Messy data loaded and attendance columns cleaned.")

Step 1: Messy data loaded and attendance columns cleaned.


In [15]:
# --- 2. LOAD HISTORY DATA FOR AUDIT ---
try:
    # Load 'season' as string to be safe
    df_history = pd.read_csv('../files/datasets/standings.csv', dtype={'season': str})

    # --- Load patch file and combine ---
    try:
        df_patch = pd.read_csv('../files/datasets/standings24_25.csv', dtype={'season': str})
        # Append patch data to history data
        df_history = pd.concat([df_history, df_patch], ignore_index=True)
        print("Step 2a: Loaded and applied 'standings24_25.csv'.")
    except FileNotFoundError:
        print("Step 2a: 'standings24_25.csv' not found. Skipping patch.")
    # --- End new patch logic ---

    print("Step 2b: Loaded historical data for auditing.")

    # Create the cleaned df_positions here for later use
    df_positions = df_history.rename(columns={
        'team_name': 'Team',
        'season': 'Season',
        'position': 'Position_History',
        'played': 'Games_Played_History',
        'wins': 'Wins_History',
        'goals_for': 'Goals_For_History',
        'goals_against': 'Goals_Against_History',
        'points': 'Points_History',
        'tier': 'Tier_History'
    })[['Team', 'Season', 'Position_History', 'Games_Played_History', 'Wins_History',
        'Goals_For_History', 'Goals_Against_History', 'Points_History', 'Tier_History']]

    # Convert new history columns to numeric
    numeric_cols = ['Position_History', 'Games_Played_History', 'Wins_History',
                    'Goals_For_History', 'Goals_Against_History', 'Points_History', 'Tier_History']
    for col in numeric_cols:
        df_positions[col] = pd.to_numeric(df_positions[col], errors='coerce')

    # --- 3. AUDIT TEAM NAMES (NOW WITH AUTO-MAPPING) ---
    # We audit the RAW, DIRTY team names from df_wide
    teams_in_wide_df = set(df_wide['Relegated Team'].astype(str).unique())
    teams_in_history_df = set(df_positions['Team'].astype(str).unique())

    unmatched_teams = teams_in_wide_df.difference(teams_in_history_df)

    # Create an empty map to build automatically
    auto_team_name_map = {}

    if unmatched_teams:
        print("--- AUDIT: TEAM NAME MISMATCHES FOUND ---")
        print("The following teams from your file do not match the history file.")
        print("Attempting to build an automatic mapping...")

        # Loop through each unique unmatched team and find the best match
        for team in sorted(unmatched_teams):
            # process.extractOne returns a tuple: (best_match, score)
            suggestion = process.extractOne(team, teams_in_history_df)

            # Auto-map spaces, asterisks, and high-confidence matches
            if suggestion[1] > 85:
                print(f"  - Auto-mapping: \"{team}\" -> \"{suggestion[0]}\" (Score: {suggestion[1]})")
                auto_team_name_map[team] = suggestion[0]
            else:
                # Don't map low-confidence or junk data
                if team not in ['nan', 'TEAM', 'Relegated Team']:
                    print(
                        f"  - WARNING: \"{team}\" has no confident match. (Best: \"{suggestion[0]}\" at {suggestion[1]}%)")
                    print(f"    -> Please add a fix for \"{team}\" to 'manual_overrides_map' in Step 4.")

        print("---------------------------------------------")
    else:
        print("--- AUDIT: TEAM NAMES OK ---")
        print("All team names in your file match the history file.")

except FileNotFoundError:
    print("Error: '../files/datasets/standings.csv' not found. Skipping audit and enrichment.")
    # Create empty dataframes if file not found
    df_positions = pd.DataFrame(columns=['Team', 'Season', 'Position_History', 'Games_Played_History',
                                         'Wins_History', 'Goals_For_History', 'Goals_Against_History',
                                         'Points_History', 'Tier_History'])
    auto_team_name_map = {}


Step 2a: Loaded and applied 'standings24_25.csv'.
Step 2b: Loaded historical data for auditing.
--- AUDIT: TEAM NAME MISMATCHES FOUND ---
The following teams from your file do not match the history file.
Attempting to build an automatic mapping...
  - Auto-mapping: " Aston Villa" -> "Aston Villa" (Score: 100)
  - Auto-mapping: " Barnsley" -> "Barnsley" (Score: 100)
  - Auto-mapping: " Birmingham City" -> "Birmingham City" (Score: 100)
  - Auto-mapping: " Blackburn Rovers" -> "Blackburn Rovers" (Score: 100)
  - Auto-mapping: " Blackpool" -> "Blackpool" (Score: 100)
  - Auto-mapping: " Bolton Wanderers" -> "Bolton Wanderers" (Score: 100)
  - Auto-mapping: " Bournemouth" -> "Bournemouth" (Score: 100)
  - Auto-mapping: " Bradford City" -> "Bradford City" (Score: 100)
  - Auto-mapping: " Burnley" -> "Burnley" (Score: 100)
  - Auto-mapping: " Cardiff City" -> "Cardiff City" (Score: 100)
  - Auto-mapping: " Charlton Athletic" -> "Charlton Athletic" (Score: 100)
  - Auto-mapping: " Coventry Ci

In [16]:
# --- 4. STANDARDIZE TEAM NAMES (THE FIX) ---
# Manual overrides for tricky names the fuzzy match might get wrong
# or for low-confidence matches found in the audit.
manual_overrides_map = {
    "Man Utd": "Manchester United",
    "Man City": "Manchester City",
    "Spurs": "Tottenham Hotspur",
    "QPR": "Queens Park Rangers",
    "Wolves": "Wolverhampton Wanderers",
    "Sheff Utd": "Sheffield United",
    "Sheff Wed": "Sheffield Wednesday",
    "Nott'm Forest": "Nottingham Forest",
    "Notts County": "Nottingham Forest"
    # e.g., if audit warns about "Bradford", add:
    # "Bradford": "Bradford City",
}

# Combine the auto-generated map with the manual overrides
# This is where we combine the two dictionaries.
# The 'manual_overrides_map' will overwrite any conflicting keys from 'auto_team_name_map'.
final_team_map = {**auto_team_name_map, **manual_overrides_map}

df_wide['Relegated Team'] = df_wide['Relegated Team'].replace(final_team_map)
print("Step 4: Standardized 'Relegated Team' names using auto-mapping and manual overrides.")

Step 4: Standardized 'Relegated Team' names using auto-mapping and manual overrides.


In [17]:
# --- 4.5. FILTERING STEP ---
# Now that names are mapped, we can filter out junk rows that didn't get mapped
teams_in_history_df = set(df_positions['Team'].astype(str).unique())
df_wide = df_wide[df_wide['Relegated Team'].isin(teams_in_history_df)]
print("Step 4.5: Filtered out junk rows (e.g., 'nan', 'TEAM').")

Step 4.5: Filtered out junk rows (e.g., 'nan', 'TEAM').


In [18]:
# --- 5. CREATE RELEGATION_EVENT_ID ---
# Now uses the CLEANED and FILTERED team names
df_wide['Relegation_Event_ID'] = df_wide['Relegated Team'] + ' ' + df_wide['Season']
print("Step 5: Created 'Relegation_Event_ID'.")

Step 5: Created 'Relegation_Event_ID'.


In [19]:
# --- Step 6: Tidying data safely ---
print("Step 6: Tidying data using robust pd.concat method...")

def make_slice(df, team_col, season_col, att_col, year_vs):
    slice_df = df.rename(columns={
        team_col: "Team",
        season_col: "Season_tmp",
        att_col: "Attendance_tmp"
    }).assign(Year_vs_Relegation=year_vs)

    # Select and rename final columns
    slice_df = slice_df[['Relegation_Event_ID', 'Team', 'Season_tmp', 'Attendance_tmp', 'Year_vs_Relegation']].copy()
    slice_df.columns = ['Relegation_Event_ID', 'Team', 'Season', 'Attendance', 'Year_vs_Relegation']

    return slice_df


# Create slices
slice_minus1 = make_slice(df_wide, "Relegated Team", "Year Before", "Year Before Att", -1)
slice_0      = make_slice(df_wide, "Relegated Team", "Season", "Attendance", 0)
slice_1      = make_slice(df_wide, "Relegated Team", "Year After", "Attendance year after", 1)
slice_2      = make_slice(df_wide, "Relegated Team", "2 years after", "Attendance 2 years after", 2)

# Concatenate slices safely
df_tidy = pd.concat([slice_minus1, slice_0, slice_1, slice_2], ignore_index=True)

print("Step 6: Data has been 'tidied' successfully!")
print(df_tidy.head())


Step 6: Tidying data using robust pd.concat method...
Step 6: Data has been 'tidied' successfully!
         Relegation_Event_ID               Team     Season Attendance  \
0  Nottingham Forest 1992-93  Nottingham Forest  1991-1992      23721   
1     Crystal Palace 1992-93     Crystal Palace  1991-1992      17618   
2      Middlesbrough 1992-93      Middlesbrough  1991-1992      14703   
3   Sheffield United 1993-94   Sheffield United  1992-1993      18801   
4    Oldham Athletic 1993-94    Oldham Athletic  1992-1993      12859   

   Year_vs_Relegation  
0                  -1  
1                  -1  
2                  -1  
3                  -1  
4                  -1  


In [20]:
dupes = df_wide.columns[df_wide.columns.duplicated()]
print("Duplicate columns:", dupes)

Duplicate columns: Index([], dtype='object')


In [21]:
# --- 7. STANDARDIZE JOIN KEY (SEASON) ---
def format_season(season_str):
    """Converts '1991-1992' to '1991-92' and leaves '1992-93' as is."""
    if pd.isna(season_str):
        return pd.NA
    parts = str(season_str).split('-')
    if len(parts) == 2:
        if len(parts[1]) == 4:  # Format is '1991-1992'
            return f"{parts[0]}-{parts[1][-2:]}"
        else:  # Format is '1992-93'
            return season_str
    return season_str  # Return as-is if not in expected format


df_tidy['Season'] = df_tidy['Season'].apply(format_season)
print("Step 7: Standardized 'Season' key to 'YYYY-YY' format (e.g., '1992-93').")

Step 7: Standardized 'Season' key to 'YYYY-YY' format (e.g., '1992-93').


In [22]:
# --- 8. ENRICH WITH POSITION DATA ---
if not df_positions.empty:

    # --- Upgrade the history file's 'Season' column ---
    # Convert 'Season' (e.g., '1992') to 'YYYY-YY' format (e.g., '1992-93')
    season_numeric = pd.to_numeric(df_positions['Season'], errors='coerce')
    season_numeric = season_numeric.dropna()  # Drop rows where 'Season' wasn't a number
    # Format the end year (e.g., 93, 00)
    season_end_year = (season_numeric + 1).astype(int).astype(str).str.zfill(4).str[-2:]
    df_positions.loc[season_numeric.index, 'Season'] = season_numeric.astype(int).astype(str) + '-' + season_end_year

    # --- Calculate Per-Game metrics using the 'played' column ---
    # This is the correct way, using the actual 'played' column
    df_positions['Goals_For_Per_Game'] = df_positions['Goals_For_History'] / df_positions['Games_Played_History']
    df_positions['Goals_Against_Per_Game'] = df_positions['Goals_Against_History'] / df_positions[
        'Games_Played_History']
    df_positions['Points_Per_Game'] = df_positions['Points_History'] / df_positions['Games_Played_History']
    df_positions['Win_Percentage'] = df_positions['Wins_History'] / df_positions['Games_Played_History']

    # Select all historical columns we need for the merge
    df_positions_merge = df_positions[['Team', 'Season', 'Position_History', 'Tier_History',
                                       'Goals_For_Per_Game', 'Goals_Against_Per_Game',
                                       'Points_Per_Game', 'Win_Percentage']]

    print("Step 8a: Upgraded history file 'Season' key and calculated Per-Game metrics.")

    # --- Merge the data ---
    print("Step 8b: Merging tidy data with position data...")
    df_final = pd.merge(
        df_tidy,
        df_positions_merge,  # Use the merge-ready dataframe
        on=['Team', 'Season'],
        how='left'
    )

    # --- Use 'Position_History' as the *only* source for 'Position' ---
    df_final['Position'] = df_final['Position_History']

    # Carry the new metrics into the main column structure
    df_final['Tier'] = df_final['Tier_History']

else:
    print("Step 8: Skipping merge as 'standings.csv' was not loaded.")
    df_final = df_tidy
    df_final['Position'] = pd.NA  # Ensure column exists
    # Ensure all columns exist even if merge skipped
    df_final['Tier'] = pd.NA
    df_final['Goals_For_Per_Game'] = pd.NA
    df_final['Goals_Against_Per_Game'] = pd.NA
    df_final['Points_Per_Game'] = pd.NA
    df_final['Win_Percentage'] = pd.NA

Step 8a: Upgraded history file 'Season' key and calculated Per-Game metrics.
Step 8b: Merging tidy data with position data...


In [26]:
# --- 9. FINAL POLISH AND SAVE ---
# This is the full list of columns for your "smart" dataset
final_columns = [
    'Relegation_Event_ID', 'Team', 'Season', 'Tier',
    'Position', 'Attendance', 'Goals_For_Per_Game', 'Goals_Against_Per_Game',
    'Points_Per_Game', 'Win_Percentage', 'Year_vs_Relegation'
]

# Ensure final columns exist before slicing (drops the temporary *_History columns)
for col in final_columns:
    if col not in df_final.columns:
        df_final[col] = pd.NA

# Attendance should be a nullable integer (Int64)
df_final['Attendance'] = pd.to_numeric(df_final['Attendance'], errors='coerce').astype('Int64')
df_final = df_final[final_columns].sort_values(by=['Relegation_Event_ID', 'Year_vs_Relegation'])

# Save the final, perfect data to a new CSV in the 'datasets' folder
output_path = "../files/datasets/perfect_tidy_data.csv"
df_final.to_csv(output_path, index=False)
import os
print("Current working directory:", os.getcwd())
print("Saving file to:", os.path.abspath(output_path))
print(f"Step 9: Success! Final table saved to '{output_path}'")

Current working directory: c:\Users\eoind\Documents\Projects\diggydigsdata.github.io\assets\scripts
Saving file to: c:\Users\eoind\Documents\Projects\diggydigsdata.github.io\assets\files\datasets\perfect_tidy_data.csv
Step 9: Success! Final table saved to '../files/datasets/perfect_tidy_data.csv'


In [24]:
# --- 10. DISPLAY THE FINAL RESULT ---
print("\n--- FINAL ENRICHED DATA TABLE ---")
print(df_final.to_markdown(index=False))


--- FINAL ENRICHED DATA TABLE ---
| Relegation_Event_ID             | Team                    | Season   |   Tier |   Position | Attendance   |   Goals_For_Per_Game |   Goals_Against_Per_Game |   Points_Per_Game |   Win_Percentage |   Year_vs_Relegation |
|:--------------------------------|:------------------------|:---------|-------:|-----------:|:-------------|---------------------:|-------------------------:|------------------:|-----------------:|---------------------:|
| Aston Villa 2015-16             | Aston Villa             | 2014-15  |      1 |         17 | 34133        |             0.815789 |                 1.5      |          1        |        0.263158  |                   -1 |
| Aston Villa 2015-16             | Aston Villa             | 2015-16  |      1 |         20 | 33690        |             0.710526 |                 2        |          0.447368 |        0.0789474 |                    0 |
| Aston Villa 2015-16             | Aston Villa             | 2016-17  |     